In [15]:
import requests
from bs4 import BeautifulSoup
import json

In [19]:
def get_song_links(url):
    """Gets song titles and URLs for the lyrics page for a www.songlyrics.com top-songs page."""
    soup = BeautifulSoup(requests.get(url).content)
    trs = soup.find_all('table', class_='tracklist')[0].find_all('tr')
    song_links = {}
    year = url[:-4]
    for row in trs[1:]:
        tds = row.find_all('td')
        artist = tds[1].get_text(strip=True)
        a_tag = tds[2].a
        song_links[(year, artist, a_tag['title'])] = a_tag['href']
    return song_links

In [20]:
def get_lyrics(url):
    """Scrapes the lyrics from a www.songlyrics.com song page."""
    soup = BeautifulSoup(requests.get(url).content, )
    return soup.find_all('p', attrs={'id':'songLyricsDiv'})[0].get_text('\n', strip=True)

In [23]:
# Define variables
base_url = 'http://www.songlyrics.com/news/top-songs/'
start_year = 2011
end_year = 1990

In [24]:
%%time
# Dict to hold song titles -> link to lyrics page.
song_links = {}
cur_year = start_year

# Loop through years and scrape links to lyric pages for each year's top 100 songs.
while cur_year >= end_year:
    url = base_url + str(cur_year)
    song_links.update(get_song_links(url))
    cur_year -= 1

ConnectionError: HTTPConnectionPool(host='www.songlyrics.com', port=80): Max retries exceeded with url: /news/top-songs/2001 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0488A550>: Failed to establish a new connection: [Errno 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [51]:
%%time
# Dict to hold song titles -> lyrics
song_lyrics = {}
errors = {}
# Visit the lyric pages for each song in song_links and scrape the lyrics.
for artist_title, url in song_links.iteritems():
    try:
        song_lyrics[artist_title] = get_lyrics(url)
    except Exception as e:
        errors[artist_title] = e

In [52]:
tup = song_lyrics.items()[2]
print(tup[0] +'\n'+ tup[1])

Who Says you can't Go Home Lyrics  Bon Jovi
I spent twenty years tryin' to get out of this place
I was lookin' for somethin' I couldn't replace
I was runnin' away from the only thing I've ever known
And like a blind dog without a bone
I was a gypsy lost in the twilight zone
I hijacked a rainbow and crashed into a pot of gold
I've been there, done that, now I ain't lookin' back
And the seeds I've sown, savin' dimes
Spendin' too much time on the telephone
Who says you can't go home?
Who says you can't go home?
There's only one place that call me one of their own
Just a hometown boy, born a rollin' stone
Who says you can't go home?
Who says you can't go back?
Been all around the world and as a matter of fact
There's only one place left, I want to go
Who says you can't go home? It's alright
It's alright, it's alright, it's alright, it's alright
I went as far as I could, I tried to find a new face
There isn't one of these lines that I would erase
I left a million mile of memories on that ro

In [11]:
len(song_lyrics)

NameError: name 'song_lyrics' is not defined

In [56]:
[k for k in song_lyrics.keys() if 'matthews' in k.lower()]

['The Space Between Lyrics  Dave Matthews Band']

In [57]:
errors

{'1 Thing Lyrics  Amerie': IndexError('list index out of range'),
 'Any Time, Any Place / And On And On Lyrics  Janet Jackson': IndexError('list index out of range'),
 'Before You Walk Out Of My Life / Like This And Like That Lyrics  Monica': IndexError('list index out of range'),
 'Big Poppa / Warning Lyrics  Notorious B.I.G.': IndexError('list index out of range'),
 'Boombastic / In The Summertime Lyrics  Shaggy': IndexError('list index out of range'),
 "C'mon And Get My Love Lyrics  D-Mob With Cathy Dennis": requests.exceptions.InvalidSchema(u"No connection adapters were found for ' http://www.songlyrics.com/d-mob-introducing-cathy-dennis/c-mon-and-get-my-love-lyrics/'"),
 'Dear Mama / Old School Lyrics Tupac': IndexError('list index out of range'),
 'Do For Love Lyrics Tupac': requests.exceptions.InvalidSchema(u"No connection adapters were found for ' http://www.songlyrics.com/tupac/do-for-love-lyrics/'"),
 "Don't Know Much Lyrics  Linda Ronstadt and Aaron Neville": IndexError('lis

In [58]:
len(errors)

35

In [ ]:
# Save song lyrics.
json.dump(song_lyrics, open('data/song_lyrics.json', 'wt'), encoding='utf8')

http://nlp.stanford.edu/IR-book/html/htmledition/document-and-query-weighting-schemes-1.html
TextBlob